# Module 0.3: 機率基礎

這個 notebook 會教你機器學習需要的機率知識。

## 學習目標

1. 理解機率的基本概念
2. 期望值和變異數
3. Gaussian 分布的性質
4. Bayes' Theorem

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

---
## Part 1: 機率分布

### 離散 vs 連續

**離散隨機變數**：只能取特定的值（如擲骰子：1, 2, 3, 4, 5, 6）
- 用 **PMF (Probability Mass Function)** 描述
- $P(X = x)$ 是機率

**連續隨機變數**：可以取任意實數值（如身高、溫度）
- 用 **PDF (Probability Density Function)** 描述
- $P(a \leq X \leq b) = \int_a^b f(x) dx$

### 為什麼 ML 要學機率？

1. **不確定性建模**：我們的預測有多「確定」？
2. **損失函數**：Cross-entropy loss 來自機率論
3. **生成模型**：VAE, GAN 等需要機率基礎
4. **貝葉斯方法**：Bayesian Neural Networks

In [ ]:
# 範例：離散分布 - 擲骰子
outcomes = [1, 2, 3, 4, 5, 6]
probabilities = [1/6] * 6  # 公平骰子

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(outcomes, probabilities, color='steelblue', edgecolor='black')
plt.xlabel('Outcome')
plt.ylabel('Probability')
plt.title('離散分布：公平骰子的 PMF')
plt.ylim(0, 0.3)

# 範例：連續分布 - Gaussian
plt.subplot(1, 2, 2)
x = np.linspace(-4, 4, 1000)
pdf = stats.norm.pdf(x, loc=0, scale=1)  # 標準常態分布
plt.plot(x, pdf, 'b-', linewidth=2)
plt.fill_between(x, pdf, alpha=0.3)
plt.xlabel('x')
plt.ylabel('Probability Density')
plt.title('連續分布：標準常態分布的 PDF')

plt.tight_layout()
plt.show()

---
## Part 2: 期望值和變異數

### 期望值 (Expectation)

「平均值」的推廣：

$$E[X] = \sum_x x \cdot P(X=x) \quad \text{(離散)}$$
$$E[X] = \int x \cdot f(x) dx \quad \text{(連續)}$$

### 變異數 (Variance)

「分散程度」的度量：

$$Var(X) = E[(X - E[X])^2] = E[X^2] - (E[X])^2$$

**標準差** $\sigma = \sqrt{Var(X)}$

### 為什麼重要？

- **MSE Loss** = $E[(y - \hat{y})^2]$
- **Batch Normalization** 需要計算 mean 和 variance

In [ ]:
def compute_mean(x):
    """
    計算樣本平均值
    
    E[X] ≈ (1/N) Σ xᵢ
    """
    # 解答：
    return np.sum(x) / len(x)


def compute_variance(x):
    """
    計算樣本變異數
    
    Var(X) ≈ (1/N) Σ (xᵢ - μ)²
    """
    # 解答：
    mean = compute_mean(x)
    return np.sum((x - mean) ** 2) / len(x)


def compute_std(x):
    """計算標準差"""
    return np.sqrt(compute_variance(x))

In [ ]:
# 測試
np.random.seed(42)
data = np.random.randn(1000)  # 1000 個標準常態樣本

print(f"我們的 mean: {compute_mean(data):.4f}")
print(f"numpy mean:  {np.mean(data):.4f}")
print()
print(f"我們的 var:  {compute_variance(data):.4f}")
print(f"numpy var:   {np.var(data):.4f}")
print()
print(f"我們的 std:  {compute_std(data):.4f}")
print(f"numpy std:   {np.std(data):.4f}")

print("\n✓ 驗證：")
assert np.isclose(compute_mean(data), np.mean(data))
assert np.isclose(compute_variance(data), np.var(data))
print("所有測試通過！")

---
## Part 3: Gaussian 分布（超重要！）

也叫「常態分布」或「高斯分布」，是 ML 中最常見的分布。

### 一維 Gaussian

$$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$$

- $\mu$：平均值（中心位置）
- $\sigma$：標準差（寬度）

### 為什麼 Gaussian 這麼常用？

1. **中央極限定理**：很多獨立隨機變數的和趨近 Gaussian
2. **最大熵原理**：在已知 mean 和 variance 時，Gaussian 是最「不確定」的分布
3. **數學方便**：很多計算有封閉解

In [ ]:
def gaussian_pdf(x, mu, sigma):
    """
    計算 Gaussian PDF
    
    f(x) = (1 / √(2πσ²)) × exp(-(x-μ)² / (2σ²))
    """
    # 解答：
    coef = 1 / np.sqrt(2 * np.pi * sigma**2)
    exponent = -((x - mu)**2) / (2 * sigma**2)
    return coef * np.exp(exponent)

In [ ]:
# 視覺化不同參數的 Gaussian
x = np.linspace(-6, 6, 1000)

plt.figure(figsize=(12, 4))

# 不同的 mean
plt.subplot(1, 2, 1)
for mu in [-2, 0, 2]:
    y = gaussian_pdf(x, mu, 1)
    plt.plot(x, y, label=f'μ={mu}, σ=1')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('改變 μ（平均值）')
plt.legend()
plt.grid(True, alpha=0.3)

# 不同的 std
plt.subplot(1, 2, 2)
for sigma in [0.5, 1, 2]:
    y = gaussian_pdf(x, 0, sigma)
    plt.plot(x, y, label=f'μ=0, σ={sigma}')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('改變 σ（標準差）')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 驗證我們的 Gaussian PDF
x_test = np.array([0, 1, -1, 2])
mu, sigma = 0, 1

our_result = gaussian_pdf(x_test, mu, sigma)
scipy_result = stats.norm.pdf(x_test, mu, sigma)

print("比較我們的實作和 scipy:")
print(f"Our:   {our_result}")
print(f"Scipy: {scipy_result}")
print(f"\n✓ Match: {np.allclose(our_result, scipy_result)}")

### 多維 Gaussian

$$f(\mathbf{x}) = \frac{1}{(2\pi)^{d/2}|\Sigma|^{1/2}} \exp\left(-\frac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^T \Sigma^{-1} (\mathbf{x}-\boldsymbol{\mu})\right)$$

- $\boldsymbol{\mu}$：平均向量
- $\Sigma$：共變異矩陣 (covariance matrix)

**直覺**：共變異矩陣決定了「橢圓」的形狀和方向。

In [ ]:
# 視覺化 2D Gaussian
from scipy.stats import multivariate_normal

# 不同的 covariance matrix
covariances = [
    np.array([[1, 0], [0, 1]]),      # 圓形
    np.array([[2, 0], [0, 0.5]]),    # 橢圓（沿軸）
    np.array([[1, 0.8], [0.8, 1]]),  # 橢圓（旋轉）
]
titles = ['獨立、相同變異', '獨立、不同變異', '相關']

x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

plt.figure(figsize=(15, 4))
for i, (cov, title) in enumerate(zip(covariances, titles)):
    plt.subplot(1, 3, i+1)
    rv = multivariate_normal([0, 0], cov)
    Z = rv.pdf(pos)
    plt.contour(X, Y, Z, levels=10)
    plt.title(f'{title}\nΣ = {cov.tolist()}')
    plt.xlabel('x₁')
    plt.ylabel('x₂')
    plt.axis('equal')

plt.tight_layout()
plt.show()

---
## Part 4: Bayes' Theorem

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

### 術語

- $P(A)$：**Prior（先驗）** - 看到 B 之前對 A 的信念
- $P(B|A)$：**Likelihood（似然）** - 如果 A 為真，看到 B 的機率
- $P(A|B)$：**Posterior（後驗）** - 看到 B 之後對 A 的更新信念

### 直覺

$$\text{後驗} \propto \text{似然} \times \text{先驗}$$

「根據新證據更新我們的信念」

### 在 ML 中的應用

- **Naive Bayes 分類器**
- **Bayesian Neural Networks**
- **最大後驗估計 (MAP)**

In [ ]:
# 經典範例：醫學檢測
# 
# 一種罕見疾病，發病率 1%
# 檢測準確率：有病檢測為陽性 99%，沒病檢測為陰性 99%
# 問：檢測陽性，實際有病的機率？

# 先驗
P_disease = 0.01  # P(D) = 1%
P_no_disease = 0.99  # P(¬D) = 99%

# 似然
P_positive_given_disease = 0.99  # P(+|D) = 99%
P_positive_given_no_disease = 0.01  # P(+|¬D) = 1% (假陽性)

# 全機率公式：P(+)
P_positive = P_positive_given_disease * P_disease + P_positive_given_no_disease * P_no_disease

# Bayes' theorem: P(D|+)
P_disease_given_positive = (P_positive_given_disease * P_disease) / P_positive

print("=" * 50)
print("醫學檢測的 Bayes' Theorem 範例")
print("=" * 50)
print(f"\n疾病發病率 P(D) = {P_disease:.2%}")
print(f"檢測敏感度 P(+|D) = {P_positive_given_disease:.2%}")
print(f"假陽性率 P(+|¬D) = {P_positive_given_no_disease:.2%}")
print(f"\n陽性機率 P(+) = {P_positive:.4f}")
print(f"\n>>> 檢測陽性時，實際有病的機率 P(D|+) = {P_disease_given_positive:.2%} <<<")
print("\n驚訝嗎？即使檢測很準確，因為疾病很罕見，")
print("大部分陽性結果仍然是假陽性！")

### 練習：實作 Naive Bayes 分類器的核心

In [ ]:
def naive_bayes_predict(x, class_priors, class_means, class_stds):
    """
    Naive Bayes 分類（假設特徵是獨立的 Gaussian）
    
    對每個類別 c，計算：
    P(c|x) ∝ P(c) × Π P(xᵢ|c)
    
    取 log 避免數值問題：
    log P(c|x) ∝ log P(c) + Σ log P(xᵢ|c)
    
    Parameters
    ----------
    x : np.ndarray, shape (n_features,)
        要分類的樣本
    class_priors : list of float
        每個類別的先驗機率 P(c)
    class_means : list of np.ndarray
        每個類別的特徵平均值
    class_stds : list of np.ndarray
        每個類別的特徵標準差
    
    Returns
    -------
    predicted_class : int
        預測的類別
    log_probs : np.ndarray
        每個類別的 log 機率（未正規化）
    """
    n_classes = len(class_priors)
    log_probs = np.zeros(n_classes)
    
    # 解答：
    for c in range(n_classes):
        # log P(c)
        log_probs[c] = np.log(class_priors[c])
        
        # 加上 Σ log P(xᵢ|c)
        # P(xᵢ|c) 是 Gaussian
        for i in range(len(x)):
            mu = class_means[c][i]
            sigma = class_stds[c][i]
            # log of Gaussian PDF
            log_probs[c] += -0.5 * np.log(2 * np.pi * sigma**2)
            log_probs[c] += -0.5 * ((x[i] - mu) / sigma)**2
    
    predicted_class = np.argmax(log_probs)
    return predicted_class, log_probs

In [ ]:
# 測試 Naive Bayes
# 假設有兩個類別，每個類別有 2 個特徵

class_priors = [0.5, 0.5]  # 相等的先驗
class_means = [
    np.array([0, 0]),  # 類別 0 的特徵平均
    np.array([2, 2]),  # 類別 1 的特徵平均
]
class_stds = [
    np.array([1, 1]),  # 類別 0 的特徵標準差
    np.array([1, 1]),  # 類別 1 的特徵標準差
]

# 測試幾個點
test_points = [
    np.array([0, 0]),   # 應該分到類別 0
    np.array([2, 2]),   # 應該分到類別 1
    np.array([1, 1]),   # 邊界附近
]

print("Naive Bayes 分類結果：")
for x in test_points:
    pred, log_probs = naive_bayes_predict(x, class_priors, class_means, class_stds)
    # 轉換成機率
    probs = np.exp(log_probs - np.max(log_probs))  # 避免溢出
    probs = probs / np.sum(probs)
    print(f"x = {x} → 類別 {pred}，機率 = {probs}")

---
## Summary

這個 notebook 你學到了：

1. **機率分布**：PMF（離散）vs PDF（連續）
2. **期望值和變異數**：描述分布的「中心」和「分散程度」
3. **Gaussian 分布**：ML 中最常見的分布，由 μ 和 σ 決定
4. **Bayes' Theorem**：用新證據更新信念

### 這些概念在 ML 中的應用：

| 概念 | 應用 |
|------|------|
| 期望值 | Loss function 的定義 |
| 變異數 | Batch Normalization |
| Gaussian | 初始化、假設、VAE |
| Bayes | Naive Bayes、Bayesian 方法 |

---

**下一個 notebook**: `04_gradient_descent.ipynb` - 梯度下降實作